In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import differential_evolution

In [2]:
#read data
d = pd.read_csv('C:/Users/cape159/Documents/pracovni/data_statistika/kopackuv_grant/mixing_experiment/DB_concept/Marstorp/marstorp1999.csv')

print(d)

       Time       CO2         Gl       DNA        CLC     Glinit   DNAinit  \
0      1.07  0.255363  49.958368  1.061615   5.166425  49.958368  1.061615   
1      1.83  0.177981        NaN       NaN        NaN  49.958368  1.061615   
2      3.11  0.239887        NaN       NaN        NaN  49.958368  1.061615   
3      4.39  0.247625        NaN       NaN        NaN  49.958368  1.061615   
4      5.92  0.263102  39.907557  1.053603  14.908990  49.958368  1.061615   
5      7.19  0.270840        NaN       NaN        NaN  49.958368  1.061615   
6      8.98  0.294055        NaN       NaN        NaN  49.958368  1.061615   
7     10.00  0.317270  30.746025  1.097670  13.295341  49.958368  1.061615   
8     11.02  0.355961        NaN       NaN        NaN  49.958368  1.061615   
9     12.05  0.386914        NaN       NaN        NaN  49.958368  1.061615   
10    13.33  0.425606        NaN       NaN        NaN  49.958368  1.061615   
11    14.35  0.464297        NaN       NaN        NaN  49.958368

In [3]:
# define DB model
def DBmodel (y, t, pars):
    #define initial states
    R=y[0];    S=y[1];    G=y[2];   DOC=y[3];   CO2=y[4]
        
    #define parameters
    Acg=pars[0];   Vmaxg=pars[1];     Kmg=pars[2] 
    Ac=pars[3];    Vmax=pars[4];      Km=pars[5]
    m0=pars[6];    f=pars[7];         Yu=pars[8]
    fs=pars[9]
            
    #G uptake rate total
    Cug_tot=Vmaxg*S*G/(G+Kmg*(1+DOC/Km))
        
    #DOC uptake rate total
    Cu_tot=Vmax*S*DOC/(DOC+Km*(1+G/Kmg))
        
    #maintnance rate total
    m_tot=m0*S
            
    #reserves mobilization rate total
    an_tot=f*R-m_tot
        
    #respiration rate
    if an_tot > 0:
        r=m_tot+an_tot*(1-Yu)+(1-Acg)*Cug_tot+(1-Ac)*Cu_tot
    else:
        r=f*R+(1-Acg)*Cug_tot+(1-Ac)*Cu_tot
             
    #derivatives
    dRdt=Acg*Cug_tot+Ac*Cu_tot-f*R
    dSdt=np.maximum(an_tot*Yu, 0)+np.minimum(0, an_tot/m0)
    dGdt=-Cug_tot
    dDOCdt=-Cu_tot-np.minimum(0, an_tot/m0*fs)
    dCO2dt=r
    
    return dRdt, dSdt, dGdt, dDOCdt, dCO2dt;

In [ ]:
#define a function returning ode results with additional calculations
def calc (model, pars, t, y0):
    #these are the model parameters
    pars1=pars[0:10]
    
    #these are the parameters to recalculate R and S to Cmic
    pars2=pars[9:12]
    
    #first solve the model
    y = odeint(model,y0,t, args=(pars1,))
    #y = pd.DataFrame(y)
    #y.columns = ['R', 'S', 'DOC', 'CO2']
    #G uptake rate total
    Cug=pars1[1]*y[:, 1]*y[:, 2]/(y[:, 2]+pars1[2]*(1+y[:, 3]/pars1[5]))
    Cu=pars1[4]*y[:, 1]*y[:, 3]/(y[:, 3]+pars1[5]*(1+y[:, 2]/pars1[2]))
          
    #calculate respiration rates and add it to y frame
    r=pd.Series([])
    for i in range(139):
        if pars1[7]*y[i, 0]-pars1[6]*y[i, 1] > 0:
            r=r.append(pd.Series(pars1[6]*y[i, 1]+(pars1[7]*y[i, 0]-pars1[6]*y[i, 1])*(1-pars1[8])+(1-pars1[0])*Cug[i]+(1-pars1[3])*Cu[i]))
        else:
            r=r.append(pd.Series(pars1[7]*y[i, 0]+(1-pars1[0])*Cug[i]+(1-pars1[3])*Cu[i]))
    
    
    #calculate Cmic and DNA, and add it to y frame
    Cmic = pars2[1] * y[:, 0] + pars2[0] * y[:, 1]
    DNA = pars2[2] * y[:, 1]
    
    yhat = np.concatenate((y[:, 3].reshape(139,1),#G
                           Cmic.reshape(139,1),
                           DNA.reshape(139,1),
                           np.array([r]).reshape(139,1)), axis=1)
    
    return yhat

In [ ]:
#create the minimization function
def obj_fun (x):
    #define parameters
    pars = x
    
    #initial conditions
    CLCinit = data.CLCinit[0]
    Ginit = data.Glinit[0]
    DOCinit = pars[12]
    Sinit = data.DNAinit[0]/pars[11]
    Rinit = (CLCinit-Sinit*pars[9])/pars[10]
        
    y0 = np.array([Rinit, Sinit, Ginit, DOCinit,0])
    
    #times
    t = data.Time
    
    #use the function to get DOC, respiration rate and Cmic
    yhat_full = calc(DBmodel, pars[0:12], t, y0)
         
    #observations
    obs=np.concatenate((np.array([data.Gl]).reshape(139,1),
                        np.array([data.CLC]).reshape(139,1),
                        np.array([data.DNA]).reshape(139,1),
                        np.array([data.CO2]).reshape(139,1)), 
                     axis=1)
    
    #weights
    weights=np.concatenate((np.nanmean(data.Gl).repeat(139).reshape(139,1),
                            np.nanmean(data.CLC/5).repeat(139).reshape(139,1),
                            np.nanmean(data.DNA/5).repeat(139).reshape(139,1),
                            np.nanmean(data.CO2).repeat(139).reshape(139,1)), 
                       axis=1)
                
          
    out=np.nansum(((yhat_full-obs)/weights)**2)
          
    return out

In [ ]:
#create goodness of fit function
def goodness (x):
    #define parameters
    pars = x
    
    #initial conditions
    CLCinit = data.CLCinit[0]
    Ginit = data.Glinit[0]
    DOCinit = pars[12]
    Sinit = data.DNAinit[0]/pars[11]
    Rinit = (CLCinit-Sinit*pars[9])/pars[10]
        
    y0 = np.array([Rinit, Sinit, Ginit, DOCinit,0])
    
    #times
    t = data.Time
    
    #use the function to get DOC, respiration rate and Cmic
    yhat_full = calc(DBmodel, pars[0:12], t, y0)
         
    #observations
    obs=np.concatenate((np.array([data.Gl]).reshape(139,1),
                        np.array([data.CLC]).reshape(139,1),
                        np.array([data.DNA]).reshape(139,1),
                        np.array([data.CO2]).reshape(139,1)), 
                     axis=1)                   
          
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = 2*13 - 2*ll
    
    out = np.array([R2, ll, AIC])
          
    return out

In [ ]:
data = d
data = data.reset_index(drop=True)
dataDOC = data.DOCinit[0]

optimuM=differential_evolution(obj_fun, [(0.05, 0.9), (0.0001, 10), (0.1, 300), 
                                         (0.05, 0.9), (0.0001, 10), (0.1, 300),
                                         (0.0001, 10), (0.0001, 10), (0.05,0.9), 
                                         (0.6, 0.8), (0.005,0.8), (0.0001,0.1), (dataDOC*1e-6, dataDOC*50)], 
                                  polish=True, maxiter=10000)

print(optimuM)

C:\Users\cape159\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in double_scalars
  app.launch_new_instance()
C:\Users\cape159\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
C:\Users\cape159\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess accuracy requested (tolerances too small). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
C:\Users\cape159\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: overflow encountered in square
C:\Users\cape159\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information

In [ ]:
print(goodness(optimuM.x))
np.savetxt('M_parameters.csv', optimuM.x.reshape(1,13), delimiter=",")